In [1]:
data = sc.textFile("s3://gmcalleh-datalake/datasets/datasets/covid19/Casos_positivos_de_COVID-19_en_Colombia-1K.csv")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1763353852166_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
data

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://gmcalleh-datalake/datasets/datasets/covid19/Casos_positivos_de_COVID-19_en_Colombia-1K.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, unix_timestamp, to_date, datediff, round

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
path_s3 = "s3://gmcalleh-datalake/datasets/datasets/covid19/Casos_positivos_de_COVID-19_en_Colombia-1K.csv"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
data_covid = spark.read.csv(path_s3, header=True, inferSchema=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
data_covid.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['fecha reporte web', 'ID de caso', 'Fecha de notificación', 'Código DIVIPOLA departamento', 'Nombre departamento', 'Código DIVIPOLA municipio', 'Nombre municipio', 'Edad', 'Unidad de medida de edad', 'Sexo', 'Tipo de contagio', 'Ubicación del caso', 'Estado', 'Código ISO del país', 'Nombre del país', 'Recuperado', 'Fecha de inicio de síntomas', 'Fecha de muerte', 'Fecha de diagnóstico', 'Fecha de recuperación', 'Tipo de recuperación', 'Pertenencia étnica', 'Nombre del grupo étnico']

In [7]:
len(data_covid.columns)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

23

In [8]:
data_covid.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1000

In [9]:
data_covid.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- fecha reporte web: string (nullable = true)
 |-- ID de caso: integer (nullable = true)
 |-- Fecha de notificación: string (nullable = true)
 |-- Código DIVIPOLA departamento: integer (nullable = true)
 |-- Nombre departamento: string (nullable = true)
 |-- Código DIVIPOLA municipio: integer (nullable = true)
 |-- Nombre municipio: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Unidad de medida de edad: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo de contagio: string (nullable = true)
 |-- Ubicación del caso: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- Código ISO del país: integer (nullable = true)
 |-- Nombre del país: string (nullable = true)
 |-- Recuperado: string (nullable = true)
 |-- Fecha de inicio de síntomas: string (nullable = true)
 |-- Fecha de muerte: string (nullable = true)
 |-- Fecha de diagnóstico: string (nullable = true)
 |-- Fecha de recuperación: string (nullable = true)
 |-- Tipo de r

In [10]:
columnas_a_eliminar = [
    'Código DIVIPOLA departamento', 
    'Código DIVIPOLA municipio', 
    'Código ISO del país', 
    'Nombre del país',
    'Nombre del grupo étnico'
]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
data_modificada = data_covid.drop(*columnas_a_eliminar)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
data_modificada.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- fecha reporte web: string (nullable = true)
 |-- ID de caso: integer (nullable = true)
 |-- Fecha de notificación: string (nullable = true)
 |-- Nombre departamento: string (nullable = true)
 |-- Nombre municipio: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Unidad de medida de edad: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo de contagio: string (nullable = true)
 |-- Ubicación del caso: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- Recuperado: string (nullable = true)
 |-- Fecha de inicio de síntomas: string (nullable = true)
 |-- Fecha de muerte: string (nullable = true)
 |-- Fecha de diagnóstico: string (nullable = true)
 |-- Fecha de recuperación: string (nullable = true)
 |-- Tipo de recuperación: string (nullable = true)
 |-- Pertenencia étnica: integer (nullable = true)

In [13]:
date_format_str = 'd/M/yyyy H:m:s'
data_modificada = data_modificada.withColumn(
    'Fecha_Notificacion', 
    to_date(col('Fecha de notificación'), date_format_str)
).withColumn(
    'Fecha_Diagnostico', 
    to_date(col('Fecha de diagnóstico'), date_format_str)
).withColumn(
    'Fecha_Inicio_Sintomas', 
    to_date(col('Fecha de inicio de síntomas'), date_format_str)
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
data_modificada = data_modificada.withColumn(
    'Dias_Diagnostico',
    datediff(col('Fecha_Diagnostico'), col('Fecha_Notificacion'))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
data_modificada.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- fecha reporte web: string (nullable = true)
 |-- ID de caso: integer (nullable = true)
 |-- Fecha de notificación: string (nullable = true)
 |-- Nombre departamento: string (nullable = true)
 |-- Nombre municipio: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Unidad de medida de edad: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo de contagio: string (nullable = true)
 |-- Ubicación del caso: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- Recuperado: string (nullable = true)
 |-- Fecha de inicio de síntomas: string (nullable = true)
 |-- Fecha de muerte: string (nullable = true)
 |-- Fecha de diagnóstico: string (nullable = true)
 |-- Fecha de recuperación: string (nullable = true)
 |-- Tipo de recuperación: string (nullable = true)
 |-- Pertenencia étnica: integer (nullable = true)
 |-- Fecha_Notificacion: date (nullable = true)
 |-- Fecha_Diagnostico: date (nullable = true)
 |-- Fecha_Inicio_Sintomas: date (

In [16]:
data_modificada.select(
    "Fecha de notificación", 
    "Fecha de diagnóstico", 
    "Fecha_Notificacion", 
    "Fecha_Diagnostico", 
    "Dias_Diagnostico"
).filter(col("Dias_Diagnostico").isNotNull()).show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+--------------------+------------------+-----------------+----------------+
|Fecha de notificación|Fecha de diagnóstico|Fecha_Notificacion|Fecha_Diagnostico|Dias_Diagnostico|
+---------------------+--------------------+------------------+-----------------+----------------+
|     2/3/2020 0:00:00|    6/3/2020 0:00:00|        2020-03-02|       2020-03-06|               4|
|     6/3/2020 0:00:00|    9/3/2020 0:00:00|        2020-03-06|       2020-03-09|               3|
|     7/3/2020 0:00:00|    9/3/2020 0:00:00|        2020-03-07|       2020-03-09|               2|
|     9/3/2020 0:00:00|   11/3/2020 0:00:00|        2020-03-09|       2020-03-11|               2|
|     9/3/2020 0:00:00|   11/3/2020 0:00:00|        2020-03-09|       2020-03-11|               2|
+---------------------+--------------------+------------------+-----------------+----------------+
only showing top 5 rows

In [18]:
datos_bogota = data_modificada.filter(
    (col('Tipo de recuperación') == 'PCR') & (col('Nombre departamento') == 'BOGOTA')
)
print(f"\nNúmero de casos con recuperación PCR en BOGOTA: {datos_bogota.count()}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Número de casos con recuperación PCR en BOGOTA: 356

In [19]:
data_modificada.select(col('Edad')).summary(
    "count", "mean", "stddev", "min", "max"
).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+
|summary|             Edad|
+-------+-----------------+
|  count|             1000|
|   mean|           42.128|
| stddev|16.90068783361342|
|    min|                1|
|    max|               88|
+-------+-----------------+

In [20]:
data_modificada.select(round("Dias_Diagnostico", 2)).describe().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------------+
|summary|round(Dias_Diagnostico, 2)|
+-------+--------------------------+
|  count|                      1000|
|   mean|                      3.57|
| stddev|        2.5006705807349685|
|    min|                         0|
|    max|                        16|
+-------+--------------------------+

In [21]:
casos_por_departamento = data_modificada.groupBy('Nombre departamento').count().alias('Total_Casos').sort(col('count').desc())
casos_por_departamento.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-----+
|Nombre departamento|count|
+-------------------+-----+
|             BOGOTA|  401|
|              VALLE|  148|
|          ANTIOQUIA|  106|
|       CUNDINAMARCA|   49|
|          CARTAGENA|   39|
|          RISARALDA|   34|
|       BARRANQUILLA|   31|
|              HUILA|   30|
|            QUINDIO|   23|
|    NORTE SANTANDER|   19|
+-------------------+-----+
only showing top 10 rows

In [22]:
casos_por_sexo_estado = data_modificada.groupBy('Sexo', 'Estado').count().alias('Total_Casos').sort(col('count').desc())
casos_por_sexo_estado.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+---------+-----+
|Sexo|   Estado|count|
+----+---------+-----+
|   M|     Leve|  486|
|   F|     Leve|  462|
|   M|Fallecido|   30|
|   F|Fallecido|   18|
|   M|      N/A|    4|
+----+---------+-----+

In [23]:
path_salida = "s3://gmcalleh-datalake/datasets/datasets/covid19/resumen-sexo-estado"

#guardar el DataFrame resultante en S3
casos_por_sexo_estado.write.csv(
    path=path_salida, 
    header=True, 
    mode="overwrite"
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…